In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from decimal import Decimal

ModuleNotFoundError: No module named 'geopandas'

In [2]:
# download daily files from: https://aqs.epa.gov/aqsweb/airdata/download_files.html#Daily
# read in each year as a pandas df

oz_pd18 = pd.read_csv("/Users/esrieves/Documents/GitHub/aq_krig/daily_44201_2018.csv")
oz_pd19 = pd.read_csv("/Users/esrieves/Documents/GitHub/aq_krig/daily_44201_2019.csv")
oz_pd20 = pd.read_csv("/Users/esrieves/Documents/GitHub/aq_krig/daily_44201_2020.csv")

In [3]:
# add in leading zeros

def rep_leading_zero(df):
    
    df["State Code"] = df["State Code"].astype(str)\
    .apply(lambda x:x.zfill(2))
    
    df["County Code"] = df["County Code"].astype(str)\
    .apply(lambda x:x.zfill(3))


    df["Site Num"] = df["Site Num"].astype(str)\
    .apply(lambda x:x.zfill(4))
    
    return df

In [4]:
oz_pd18 = rep_leading_zero(oz_pd18)
oz_pd19 = rep_leading_zero(oz_pd19)
oz_pd20 = rep_leading_zero(oz_pd20)

In [5]:
# drop all obs for non Denver area sites
# Adams = 001, Arapahoe = 005, Boulder = 013, Broomfield = 014, Denver = 031, Douglas = 035, Jefferson = 059, Weld = 123, EL Paso = 041, Larimer = 069

def drop_nonFOCO(df):
    den_metro_counties = ["001","005","013","014","031","059"]
    df = df[(df["State Code"] == "08") & (df["County Code"].isin(den_metro_counties))]
    return df

In [6]:
oz_pd18 = drop_nonFOCO(oz_pd18)
oz_pd19 = drop_nonFOCO(oz_pd19)
oz_pd20 = drop_nonFOCO(oz_pd20)

In [7]:
# Function to check that aspects like parameter, sample duration, events, datum, etc. are what is expected

def check_consistency(df):
    cons_frame = []
    n_param_code = np.unique(df["Parameter Code"])
    n_sites = len(np.unique(df["Site Num"]))
    n_param_name = np.unique(df["Parameter Name"])
    n_sample_dur = np.unique(df["Sample Duration"])
    n_poll_std = np.unique(df["Pollutant Standard"])
    n_units = np.unique(df["Units of Measure"])
    n_event = len(df[df["Event Type"] != "None"])
    n_poc = np.unique(df["POC"])
    n_datum = np.unique(df["Datum"])
    cons_frame.append([n_param_code,n_param_name,n_sites,n_sample_dur,n_poll_std,n_units,n_event,n_poc,n_datum])
    return cons_frame


In [8]:
check_consistency(oz_pd18)

# everything as expected, no events (many more events when adding new counties), two datums (req trans), indicates duplicated dates for a site

[[array([44201]),
  array(['Ozone'], dtype=object),
  8,
  array(['8-HR RUN AVG BEGIN HOUR'], dtype=object),
  array(['Ozone 8-hour 2015'], dtype=object),
  array(['Parts per million'], dtype=object),
  0,
  array([1, 2, 6]),
  array(['NAD83', 'WGS84'], dtype=object)]]

In [9]:
test = oz_pd18[oz_pd18["Event Type"] != "None"]
test[["Arithmetic Mean","Event Type"]]

,Arithmetic Mean,Event Type


In [10]:
check_consistency(oz_pd19)

# everything as expected, no events, two datums (req trans), indicates duplicated dates for a site

[[array([44201]),
  array(['Ozone'], dtype=object),
  8,
  array(['8-HR RUN AVG BEGIN HOUR'], dtype=object),
  array(['Ozone 8-hour 2015'], dtype=object),
  array(['Parts per million'], dtype=object),
  0,
  array([1, 2, 6]),
  array(['NAD83', 'WGS84'], dtype=object)]]

In [11]:
test = oz_pd19[oz_pd19["Event Type"] != "None"]
test[["Arithmetic Mean","Event Type"]]

,Arithmetic Mean,Event Type


In [12]:
check_consistency(oz_pd20)

# everything as expected, no events, two datums (req trans), indicates duplicated dates for a site

[[array([44201]),
  array(['Ozone'], dtype=object),
  7,
  array(['8-HR RUN AVG BEGIN HOUR'], dtype=object),
  array(['Ozone 8-hour 2015'], dtype=object),
  array(['Parts per million'], dtype=object),
  0,
  array([1, 2, 6]),
  array(['NAD83', 'WGS84'], dtype=object)]]

In [13]:
test = oz_pd20[oz_pd20["Event Type"] != "None"]
test[["Arithmetic Mean","Event Type"]]

,Arithmetic Mean,Event Type


In [8]:
# remove unnecessary columns (includes units, event type because confirmed to be consistent/unimportant earlier)

def drop_noise(df):
    df = df.drop(columns=["Parameter Code", "Parameter Name", "Sample Duration", "Pollutant Standard", 
                          "Observation Count", "1st Max Value", "Units of Measure", "1st Max Hour", "AQI",
                          "Method Code", "Method Name", "Address", "City Name", "CBSA Name", "Date of Last Change",
                          "State Name", "County Name", "Event Type"])
    return df


In [9]:
oz_pd18 = drop_noise(oz_pd18)
oz_pd19 = drop_noise(oz_pd19)
oz_pd20 = drop_noise(oz_pd20)

In [10]:
# remove observations with less than 80% completeness

def full_obs(df):
    df = df[df["Observation Percent"] > 79]
    return df

In [11]:
oz_pd18 = full_obs(oz_pd18)
oz_pd19 = full_obs(oz_pd19)
oz_pd20 = full_obs(oz_pd20)

In [12]:
# rename columns

def rename_cols(df):
    df = df.rename(columns={"State Code": "state_code", "County Code": "county_code", "Site Num": "site_num",
                            "Latitude":"lat","Longitude":"long","Datum":"datum","Date Local":"date",
                            "Observation Percent":"obs_perc","Arithmetic Mean":"mda8","Local Site Name":"site_name"})
    return df

In [13]:
oz_pd18 = rename_cols(oz_pd18)
oz_pd19 = rename_cols(oz_pd19)
oz_pd20 = rename_cols(oz_pd20)

In [14]:
# turns df into a geopandas object with geometry,
# transforms NAD83 to WGS84
# standardizes datum

def datum_transform(df):
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.long,df.lat))
    gdf = gdf.set_crs("EPSG:4326")
    return gdf


In [15]:
oz_pd18 = datum_transform(oz_pd18)
oz_pd19 = datum_transform(oz_pd19)
oz_pd20 = datum_transform(oz_pd20)

In [141]:
oz_pd18["UniqueID"] = oz_pd18["state_code"]+"_"+oz_pd18["county_code"]+"_"+oz_pd18["site_num"]
oz_pd19["UniqueID"] = oz_pd19["state_code"]+"_"+oz_pd19["county_code"]+"_"+oz_pd19["site_num"]
oz_pd20["UniqueID"] = oz_pd20["state_code"]+"_"+oz_pd20["county_code"]+"_"+oz_pd20["site_num"]

In [16]:
o18 = oz_pd18.drop(columns=["state_code","county_code","site_num","POC","obs_perc","datum"])
o19 = oz_pd19.drop(columns=["state_code","county_code","site_num","POC","obs_perc","datum"])
o20 = oz_pd20.drop(columns=["state_code","county_code","site_num","POC","obs_perc","datum"])

In [17]:
o3 = pd.concat([o18,o19,o20],axis=0)
#o3 = o3.set_index("UniqueID")

In [18]:
# create date as a datetime object
o3["date"] = pd.to_datetime(o3["date"])

In [19]:
# group by site
o3_gr = o3.groupby("site_name")


# resample grouped sites for weekly and monthly mean
o3_mthly_avg = o3_gr.resample("M", on="date").mean()
o3_wkly_avg = o3_gr.resample("W",on="date").mean()


# reset index
o3_mthly_avg = o3_mthly_avg.reset_index()
o3_wkly_avg = o3_wkly_avg.reset_index()


o3_mthly_avg

,site_name,date,lat,long,mda8
0,Aspen Park,2018-01-31,39.541515,-105.29841,0.036137
1,Aspen Park,2018-02-28,39.541515,-105.29841,0.037181
2,Aspen Park,2018-03-31,39.541515,-105.29841,0.042224
3,Aspen Park,2018-04-30,39.541515,-105.29841,0.046615
4,Aspen Park,2018-05-31,39.541515,-105.29841,0.045023
...,...,...,...,...,...
343,Welby,2020-08-31,39.838119,-104.94984,0.046063
344,Welby,2020-09-30,39.838119,-104.94984,0.032284
345,Welby,2020-10-31,39.838119,-104.94984,0.023508
346,Welby,2020-11-30,39.838119,-104.94984,0.019069


In [63]:
# correct lat/long rounding error
o3_mthly_avg["lat"] = round(o3_mthly_avg["lat"],16)
o3_mthly_avg["long"] = round(o3_mthly_avg["long"],16)

In [64]:
o3_pivot = o3_mthly_avg.pivot(index=["site_name","lat","long"],
             columns="date",
             values="mda8")

In [65]:
o3_pivot

,,date,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,...,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31
site_name,lat,long,,,,,,,,,,,,,,,,,,,,,
Aspen Park,39.541515,-105.298410,0.036137,0.037181,0.042224,0.046615,0.045023,0.045673,0.050725,0.048015,0.039641,0.029714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aurora East,39.638522,-104.569335,0.033914,0.035423,0.043284,0.045437,0.047594,0.049613,0.053108,0.049241,0.039178,0.030839,...,0.038948,0.043688,0.043446,0.045871,0.046229,0.052292,0.042189,0.034698,0.032733,0.031859
Boulder Reservoir,40.070016,-105.220238,0.025405,0.033185,0.039260,0.043205,0.045024,0.047429,0.051867,0.050076,0.039282,0.029572,...,0.035385,0.042572,0.042624,0.044008,0.045945,0.051041,0.039218,0.030742,0.029111,0.027928
DENVER - CAMP,39.751184,-104.987625,0.014378,0.018618,0.030823,0.037638,0.039232,0.042099,0.047169,0.043549,0.031023,0.018980,...,0.028455,0.035505,0.037679,0.038688,0.042280,0.047258,0.033141,0.022778,0.019191,0.018321
Evergreen,39.620408,-105.338720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.030422,0.030220,0.033728
HIGHLAND RESERVOIR,39.567887,-104.957193,0.026758,0.026815,0.040985,0.045898,0.048276,0.050829,0.054471,0.049638,0.039974,0.028101,...,0.036704,0.044278,0.045742,0.046651,0.048250,0.055744,0.043147,0.033532,0.031759,0.030780
La Casa,39.779490,-105.005180,0.015135,0.020855,0.031345,0.037246,0.040021,0.043034,0.046811,0.044351,0.031377,0.018491,...,0.027717,0.036373,0.039156,0.042235,0.043570,0.047047,0.031836,0.024359,0.019344,0.017187
NATIONAL RENEWABLE ENERGY LABS - NREL,39.743724,-105.177989,0.032392,0.032376,0.041222,0.043743,0.045844,0.049237,0.051956,0.054891,0.045056,0.030755,...,0.035994,0.042912,0.047226,0.048853,0.050152,0.060536,0.047878,0.036657,0.035910,0.035136
ROCKY FLATS-N,39.912799,-105.188587,0.036336,0.036589,0.044290,0.047426,0.049741,0.052248,0.056920,0.055418,0.044878,0.032082,...,0.039090,0.046992,0.045822,0.049421,0.049972,0.058266,0.050328,0.037969,0.037578,0.036373


In [67]:
o3_pivot.to_csv("o3_mthly_wide.csv")

In [84]:
o3.to_csv("o3.csv")

In [35]:
o3_mthly_avg.to_csv("o3_mthly_avg.csv")

In [29]:
o3_wkly_avg.to_csv("o3_wkly_avg.csv")